In [17]:
import spotipy.util as util 
import json

# Accessing the Spotify API

In [18]:
def load_config(json_file):
    '''
    This function takes a json file path as input and returns all the
    configuration variables needed: username, client_id, client_secret
    redirect_uri and scope in that order.
    '''
    with open(json_file, "r") as config_file:
        # Load json config file
        data = json.load(config_file)
        # Extract config variables
        username = data['username']
        client_id = data['client_id']
        client_secret = data['client_secret']
        redirect_uri = data['redirect_uri']
        scope = data['scope']
    
    return username, client_id, client_secret, redirect_uri, scope

In [14]:
username, client_id, client_secret, redirect_uri, scope = load_config("config.json")

token = util.prompt_for_user_token(username=username,
                                  client_id=client_id,
                                  client_secret=client_secret,
                                  redirect_uri=redirect_uri)

Spotify does not include the track ID's and therefore we need to define a function in order to retrieve them:

In [4]:
import requests
def get_id(track_name: str, token: str) -> str:
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer ' + token,
    }
    params = [
    ('q', track_name),
    ('type', 'track'),
    ]
    try:
        response = requests.get('https://api.spotify.com/v1/search', 
                    headers = headers, params = params, timeout = 5)
        json = response.json()
        first_result = json['tracks']['items'][0]
        track_id = first_result['id']
        return track_id
    except:
        return None

## Testing our function:

In [16]:
lucy_id = get_id('Hey Jude', token)
print(lucy_id)

0aym2LBJBk9DAYuHHutrIl


## Getting the features

In [5]:
def get_features(track_id: str, token: str) -> dict:
    sp = spotipy.Spotify(auth=token)
    try:
        features = sp.audio_features([track_id])
        return features[0]
    except:
        return None

In [19]:
lucy_features = get_features(lucy_id, token)
print(lucy_features)

{'danceability': 0.386, 'energy': 0.607, 'key': 10, 'loudness': -7.7, 'mode': 1, 'speechiness': 0.0261, 'acousticness': 0.0112, 'instrumentalness': 1.38e-05, 'liveness': 0.088, 'valence': 0.532, 'tempo': 147.207, 'type': 'audio_features', 'id': '0aym2LBJBk9DAYuHHutrIl', 'uri': 'spotify:track:0aym2LBJBk9DAYuHHutrIl', 'track_href': 'https://api.spotify.com/v1/tracks/0aym2LBJBk9DAYuHHutrIl', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0aym2LBJBk9DAYuHHutrIl', 'duration_ms': 425653, 'time_signature': 4}


## Get streamings

In [6]:
import ast
import requests
from datetime import datetime
from typing import List
import spotipy
import spotipy.util as util
from os import listdir
import pandas as pd
import history 

def get_streamings(path: str = 'MyData', 
                ) -> List[dict]:
    
    '''Returns a list of streamings form spotify MyData dump.
    Will not acquire track features.'''
    
    files = ['MyData/' + x for x in listdir(path)
             if x.split('.')[0][:-1] == 'StreamingHistory']
    
    all_streamings = []
    
    for file in files: 
        with open(file, 'r', encoding='UTF-8') as f:
            new_streamings = ast.literal_eval(f.read())
            all_streamings += [streaming for streaming in new_streamings]
            
    #adding datetime field
    for streaming in all_streamings:
        streaming['datetime'] = datetime.strptime(streaming['endTime'], '%Y-%m-%d %H:%M')    
    return all_streamings

In [7]:
streamings = get_streamings()

# Building our streaming history

In [ ]:
streamings = get_streamings()
unique_tracks = list(set([streaming['trackName']
                for streaming in streamings]))

all_features = {}
for track in unique_tracks:
    track_id = get_id(track, token)
    features = get_features(track_id, token)
    
    if features:
        all_features[track] = features

with_features = []

for track_name, features in all_features.items():
    with_features.append({'name':track_name, **features})

This marks the end of the Feature Extraction process for this project.